In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

%matplotlib inline

In [2]:
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
    "timestamp": x["timestamp"]
})
movies = movies.map(lambda x: x["movie_title"])

C:\Users\ujwal\Twin_tower_model\twin_model\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|                                                                         | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|                                                                              | 0/4 [00:01<?, ? MiB/s]

Extraction completed...: 0 file [00:01, ? file/s]
Dl Size...:  25%|█████████████████▌                                                    | 1/4 [00:05<00:16,  5.34s/ MiB]

Extraction completed...: 0 file [00:05, ? file/s]
Dl Size...:  50%|███████████████████████████████████                                   | 2/4 [00:07<00:07,  3.71s/ MiB]

Extraction completed...: 0 file [00:07,

Dataset movielens downloaded and prepared to C:\Users\ujwal\tensorflow_datasets\movielens\100k-ratings\0.1.1. Subsequent calls will reuse this data.


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|                                                                         | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 42.69 url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|                                                                        | 0/4924029 [00:00<?, ? MiB/s]

Dl Size...: 100%|████████████████████████████████████████████████████| 4924029/4924029 [00:00<00:00, 86036436.59 MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]
Generating splits...:   0%|                                                                 | 0/1 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
                                                              
Shuffling C:\Users\ujwal\tensorflow_datasets\movielens\100k-movies\incompl

Dataset movielens downloaded and prepared to C:\Users\ujwal\tensorflow_datasets\movielens\100k-movies\0.1.1. Subsequent calls will reuse this data.


In [3]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [4]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(
    np.concatenate(
        list(ratings.batch(1_000).map(lambda x: x["user_id"]))
    )
)